In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.optimizers import RMSprop

In [3]:
dataset = keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
(x_train,y_train),(x_test,y_test) = dataset
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [5]:
x_train = x_train.reshape(-1,28,28,1)

In [6]:
# Preprocess the data (these are NumPy arrays)
# x_train = x_train.reshape(60000, 784).astype("float32") / 255
# x_test = x_test.reshape(10000, 784).astype("float32") / 255

x_train = x_train / 255
x_test = x_test / 255

# y_train = y_train.astype("float32")
# y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [7]:
# #create a functional API
# inputs = keras.Input(shape=(28,28,1), name="digits")
# # layer= layers.Conv2D(8,3, activation='relu',name="ConvLayer0")(inputs)
# layer = layers.Conv2D(16,3, activation='relu',name="ConvLayer1")(inputs)
# layer= layers.Conv2D(32,3, activation='relu',name="ConvLayer2")(layer)
# layer= layers.Conv2D(64,3, activation='relu',name="ConvLayer3")(layer)
# layer= layers.Conv2D(128,3, activation='relu',name="ConvLayer4")(layer)
# layer = layers.MaxPooling2D(3)(layer)
# layer= layers.Conv2D(256,3, activation='relu',name="ConvLayer5")(layer)
# layer = layers.MaxPooling2D(3)(layer)
# layer = layers.Flatten()(layer)
# layer = layers.Dense(256,activation = 'relu',name="DenseLayer1")(layer)
# # layer = layers.Dense(128,activation = 'relu',name="DenseLayer2")(layer)
# layer = layers.Dense(64,activation = 'relu',name="DenseLayer3")(layer)
# pred = layers.Dense(10,activation = 'softmax',name="prediction_layer")(layer)

In [8]:
#create a functional API
inputs = keras.Input(shape=(28,28,1), name="digits")
layer= layers.Conv2D(32,5, activation='relu',name="ConvLayer2")(inputs)
layer= layers.Conv2D(32,5, activation='relu',name="ConvLayer3")(layer)
layer = layers.MaxPooling2D(2)(layer)
layer = layers.Dropout(0.25)(layer)

layer= layers.Conv2D(64,3, activation='relu',name="ConvLayer4")(layer)
layer= layers.Conv2D(64,3, activation='relu',name="ConvLayer5")(layer)
# layer = layers.MaxPooling2D(2)(layer)
layer = layers.Dropout(0.25)(layer)

layer= layers.Conv2D(128,3, activation='relu',name="ConvLayer6")(layer)
layer= layers.Conv2D(128,3, activation='relu',name="ConvLayer7")(layer)
layer = layers.MaxPooling2D(2)(layer)
layer = layers.Dropout(0.25)(layer)

layer = layers.Flatten()(layer)
layer = layers.Dense(256,activation = 'relu',name="DenseLayer1")(layer)
layer = layers.Dropout(0.25)(layer)
layer = layers.Dense(128,activation = 'relu',name="DenseLayer2")(layer)
layer = layers.Dropout(0.5)(layer)
pred = layers.Dense(10,activation = 'softmax',name="prediction_layer")(layer)

In [9]:
model=keras.Model(inputs=inputs,outputs=pred)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 28, 28, 1)]       0         
_________________________________________________________________
ConvLayer2 (Conv2D)          (None, 24, 24, 32)        832       
_________________________________________________________________
ConvLayer3 (Conv2D)          (None, 20, 20, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 10, 10, 32)        0         
_________________________________________________________________
ConvLayer4 (Conv2D)          (None, 8, 8, 64)          18496     
_________________________________________________________________
ConvLayer5 (Conv2D)          (None, 6, 6, 64)         

In [10]:
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [11]:
datagen = keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        data_format = None,
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train) #,target_size=(28,28), batch_size=32, frames_per_step=4)

In [12]:
model.compile(optimizer = 'adam', 
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy(),keras.metrics.MeanAbsoluteError()])

model.fit_generator(datagen.flow(x_train,y_train, batch_size=32),epochs=50,validation_data=(x_val,y_val),callbacks= [learning_rate_reduction])

model.save("digit_regonizer_best_model.h5")

results = model.evaluate(x_test,y_test,batch_size=64)

Epoch 1/50
1563/1563 [==============================] - 20s 13ms/step - loss: 0.5473 - sparse_categorical_accuracy: 0.8295 - mean_absolute_error: 4.3686 - val_loss: 0.0675 - val_sparse_categorical_accuracy: 0.9813 - val_mean_absolute_error: 4.3992
Epoch 2/50
1563/1563 [==============================] - 21s 13ms/step - loss: 0.1545 - sparse_categorical_accuracy: 0.9622 - mean_absolute_error: 4.3686 - val_loss: 0.0504 - val_sparse_categorical_accuracy: 0.9872 - val_mean_absolute_error: 4.3992
Epoch 3/50
1563/1563 [==============================] - 20s 13ms/step - loss: 0.1209 - sparse_categorical_accuracy: 0.9717 - mean_absolute_error: 4.3686 - val_loss: 0.0531 - val_sparse_categorical_accuracy: 0.9853 - val_mean_absolute_error: 4.3992
Epoch 4/50
1563/1563 [==============================] - 19s 12ms/step - loss: 0.1068 - sparse_categorical_accuracy: 0.9749 - mean_absolute_error: 4.3686 - val_loss: 0.0457 - val_sparse_categorical_accuracy: 0.9879 - val_mean_absolute_error: 4.3992
Epoch 5/

In [13]:
# train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")
sample = pd.read_csv("../input/digit-recognizer/sample_submission.csv")

# print(train.shape)
print(test.shape)
print(sample.shape)

(28000, 784)
(28000, 2)


In [14]:
test= np.array(test)
test = test.reshape(28000, 28, 28) / 255
print(test.shape)

(28000, 28, 28)


In [15]:
# new_x_test = np.array(test)
# new_x_test = new_x_test/255
predictions = model.predict(test)
for i in range(0,len(predictions)):
    sample.loc[i]['Label'] = tf.argmax(predictions[i])
sample.to_csv("sample_submission.csv",index=False, header=True)

In [16]:
sample

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
